In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
i=1
r = requests.get(f"https://www.loc.gov/pictures/search/?q=mrg&fa=displayed%3Aanywhere&sp={1}&st=gallery")

In [ ]:
soup = BeautifulSoup(r.text,"html.parser")

In [ ]:
class_first = soup.find_all("td",class_="first")

In [ ]:
resource_link = class_first[0].find("a")["href"].strip()

In [ ]:
def get_page_soup(page_number:int):
    r = requests.get(f"https://www.loc.gov/pictures/search/?q=mrg&fa=displayed%3Aanywhere&sp={page_number}&st=gallery")
    soup = BeautifulSoup(r.text,"html.parser")
    return soup

In [ ]:
def find_resource_links_on_page(soup,links:list):
    class_first = soup.find_all("td")
    for i,j in enumerate(class_first):
        resource_link = class_first[i].find("a")["href"].strip()
        links.append(resource_link)
        

In [ ]:
from tqdm import tqdm
links = []
for i in tqdm(range(1,587)):
    find_resource_links_on_page(get_page_soup(i),links)

In [ ]:
# import json
# with open("links","w") as f:
#     json.dump(links,f)

In [ ]:
r = requests.get(links[0])

In [ ]:
soup=BeautifulSoup(r.text,"html.parser").find("div",id="resource")

In [ ]:
soup.find("img")["alt"],soup.find("img")["src"]

In [ ]:
import json
with open("links","r") as  f:
    links = json.load(f)

In [ ]:
links[0]

In [ ]:
def get_img_tuples(link:str):
    r = requests.get(link)
    soup=BeautifulSoup(r.text,"html.parser").find("div",id="resource")
    app = (soup.find("img")["alt"],soup.find("img")["src"])
    img_text_tuples.append(app)

In [ ]:
from tqdm import tqdm
import requests
img_text_tuples = []
for link in tqdm(links[len(img_text_tuples):]):
    get_img_tuples(link)

In [ ]:
import json
with open("resources","w") as f:
    json.dump(img_text_tuples,f)

In [ ]:
img_text_tuples[0]

In [3]:
import pandas as pd
import json

In [4]:
with open("resources","r") as f:
    img_text_tuples = json.load(f)

In [5]:
df = pd.DataFrame(img_text_tuples,columns=["Location","link"])

In [6]:
def get_city(string):
    return ", ".join(string.split(",")[::-1][0:2][::-1])

In [7]:
df["city"] = df["Location"].apply(get_city)

In [25]:
df.

,Location,link,city
0,"Teapot Dome gas station, Zillah, Washington",//cdn.loc.gov/service/pnp/mrg/00000/00001r.jpg,"Zillah, Washington"
1,"Harold's Auto Center, horizontal view, Sinclai...",//cdn.loc.gov/service/pnp/mrg/00000/00002r.jpg,"Spring Hill, Florida"
2,Premium gas station (formerly Joy gas station)...,//cdn.loc.gov/service/pnp/mrg/00000/00003r.jpg,"Ontario, Canada"
3,"Bomber gas station, diagonal view, Route 99 E....",//cdn.loc.gov/service/pnp/mrg/00000/00004r.jpg,"Milwaukie, Oregon"
4,"Hat n' Boots gas station, overall view, Route ...",//cdn.loc.gov/service/pnp/mrg/00000/00005r.jpg,"Seattle, Washington."


In [9]:
city_set = set(df.city.values)

In [19]:
import googlemaps
from datetime import datetime
from decouple import config
from tqdm import tqdm

city_dict = {}
for city in tqdm(list(city_set)):
    API_KEY = config("API_KEY")
    import requests

    url = f"""https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={city}&inputtype=textquery&fields=geometry&key={API_KEY}"""

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    city_dict[city]=response.json()

100%|██████████████████████████████████████████████████████████████████████████████| 1945/1945 [13:40<00:00,  2.37it/s]


In [22]:
# with open("lat_lon.json","w") as f:
#     json.dump(city_dict,f)

In [51]:
def get_geometry(city):
    try:
        return city_dict[city]["candidates"][0]["geometry"]["location"]
    except:
        return None
[get_geometry(i) for i in df.city]

[{'lat': 46.4020734, 'lng': -120.2619988},
 {'lat': 28.4831682, 'lng': -82.5369872},
 {'lat': 51.253775, 'lng': -85.323214},
 {'lat': 45.4465305, 'lng': -122.6322812},
 {'lat': 47.6062095, 'lng': -122.3320708},
 {'lat': 47.6062095, 'lng': -122.3320708},
 {'lat': 39.4603448, 'lng': -123.258793},
 {'lat': 36.09985959999999, 'lng': -80.244216},
 {'lat': 38.9716689, 'lng': -95.2352501},
 {'lat': 43.0389025, 'lng': -87.9064736},
 {'lat': 37.8066882, 'lng': -100.3482003},
 {'lat': 43.2128473, 'lng': -75.4557303},
 {'lat': 32.2226066, 'lng': -110.9747108},
 {'lat': 42.5972563, 'lng': -82.8779754},
 {'lat': 30.7999563, 'lng': -81.68982609999999},
 {'lat': 34.7400809, 'lng': -97.22224510000001},
 {'lat': 42.0600834, 'lng': -75.4276769},
 {'lat': 42.0600834, 'lng': -75.4276769},
 {'lat': 40.7127753, 'lng': -74.0059728},
 {'lat': 41.7003713, 'lng': -73.9209701},
 {'lat': 41.9270367, 'lng': -73.9973608},
 {'lat': 34.1161196, 'lng': -118.1503488},
 {'lat': 41.394817, 'lng': -73.4540111},
 {'lat': 3

In [55]:
df.insert(3,"geometry",[get_geometry(i) for i in df.city])

In [57]:
df.to_json("df.json")